In [1]:
# !pip install -U transformers accelerate datasets git+https://github.com/huggingface/diffusers Pillow==9.4.0 torchmetrics wandb

In [2]:
# from local_secrets import hf_token, wandb_key
# from huggingface_hub import login
# import wandb

# login(token=hf_token)
# wandb.login(key=wandb_key)

In [3]:
import torch, torch.nn.functional as F, random, wandb, time
import torchvision.transforms as T
from torchvision import transforms
from diffusers import AutoencoderDC, SanaTransformer2DModel
from diffusers.schedulers import FlowMatchEulerDiscreteScheduler
from transformers import AutoModel, AutoTokenizer, set_seed
from datasets import load_dataset, Dataset, DatasetDict
from tqdm import tqdm
from torch.utils.data import DataLoader

from utils import PIL_to_latent, latent_to_PIL, make_grid, encode_prompt, dcae_scalingf, pil_clipscore
from utils import cifar10_labels

seed = 42
set_seed(seed)

In [4]:
dtype = torch.bfloat16
device = "cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu"

transformer = SanaTransformer2DModel.from_config("transformer_Sana-7L-MBERT_config.json").to(device).to(dtype)
text_encoder = AutoModel.from_pretrained("answerdotai/ModernBERT-base", torch_dtype=dtype).to(device)
tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base", torch_dtype=dtype)

model = "Efficient-Large-Model/Sana_600M_1024px_diffusers"
dcae = AutoencoderDC.from_pretrained(model, subfolder="vae", torch_dtype=dtype).to(device)
scheduler = FlowMatchEulerDiscreteScheduler.from_pretrained(model, subfolder="scheduler")

/home/g/.local/lib/python3.10/site-packages/diffusers/configuration_utils.py:245: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a model, please use <class 'diffusers.models.transformers.sana_transformer.SanaTransformer2DModel'>.load_config(...) followed by <class 'diffusers.models.transformers.sana_transformer.SanaTransformer2DModel'>.from_config(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


# Load dataset

In [5]:
ds = load_dataset("g-ronimo/CIFAR10-64-latents_dc-ae-f32c32-sana-1.0")
ds["train"]

Dataset({
    features: ['label', 'latent'],
    num_rows: 50000
})

In [6]:
labels = cifar10_labels
labels_encoded={k: encode_prompt(labels[k], tokenizer, text_encoder) for k in labels}

len(labels_encoded), len(labels_encoded[0]), labels_encoded[0][0].shape, labels_encoded[0][1].shape

(10, 2, torch.Size([1, 300, 768]), torch.Size([1, 300]))

In [7]:
from torch.utils.data import DataLoader

def collate(items):
    labels = [i["label"] for i in items]
    latents = torch.cat([torch.Tensor(i["latent"]) for i in items]).to(dtype).to(device)
    prompts_encoded = torch.cat([labels_encoded[label][0] for label in labels])
    prompts_atnmask = torch.cat([labels_encoded[label][1] for label in labels])

    return labels, latents, prompts_encoded, prompts_atnmask

dataloader = DataLoader(ds["train"], batch_size=2, shuffle=True, generator = torch.manual_seed(seed), collate_fn=collate)
labels, latents, prompts_encoded, prompts_atnmask = next(iter(dataloader))
len(labels), latents.mean(), latents.shape, prompts_encoded.shape, prompts_atnmask.shape

(2,
 tensor(-0.0366, device='cuda:0', dtype=torch.bfloat16),
 torch.Size([2, 32, 2, 2]),
 torch.Size([2, 300, 768]),
 torch.Size([2, 300]))

# Helpers for eval and generate

In [8]:
scheduler.set_timesteps(100)

In [9]:
def generate(prompt, latent_dim=[1, 32, 2, 2], latent_seed=42):
    scheduler.set_timesteps(scheduler.timesteps.size(0))     # reset step index
    prompt_encoded, prompt_atnmask = encode_prompt(prompt, tokenizer, text_encoder)
    latents = torch.randn(latent_dim, generator = torch.manual_seed(latent_seed)).to(dtype).to(device)

    for t_idx in range(scheduler.timesteps.size(0)):
        t = scheduler.timesteps[t_idx].unsqueeze(0).to(device)
        with torch.no_grad():
            noise_pred = transformer(latents, encoder_hidden_states=prompt_encoded, timestep=t, encoder_attention_mask=prompt_atnmask, return_dict=False)[0]
        latents = scheduler.step(noise_pred, t, latents, return_dict=False)[0]
    return latent_to_PIL(latents / dcae_scalingf, dcae)

[generate("0")]

[<PIL.Image.Image image mode=RGB size=64x64>]

In [10]:
def eval_loss(data_val, num_samples=10, batch_size=24):
    losses = []
    eval_dataloader = iter(DataLoader(data_val, batch_size=batch_size, shuffle=False, collate_fn=collate))
    
    for i in tqdm(range(num_samples), "eval_loss"):
        label, latent, prompt_encoded, prompt_atnmask = next(eval_dataloader)
        noise = torch.randn_like(latent)
        timestep = scheduler.timesteps[torch.randint(scheduler.timesteps.size(0),(latent.shape[0],))].to(device)
        latent_noisy = scheduler.scale_noise(latent, timestep, noise)
        with torch.no_grad():
            noise_pred = transformer(latent_noisy, encoder_hidden_states = prompt_encoded, encoder_attention_mask = prompt_atnmask, timestep = timestep, return_dict=False)[0]
        loss = F.mse_loss(noise_pred, noise - latent)
        losses.append(loss.item())
    return sum(losses)/len(losses)

eval_loss(ds["train"])

eval_loss: 100%|████████████████████████████████| 10/10 [00:00<00:00, 53.96it/s]


6.659375

In [11]:
def eval_clipscore(images):
    prompts = [cifar10_labels[k] for k in cifar10_labels]
    return pil_clipscore(images, prompts)

scheduler.set_timesteps(100)
images = [generate(p) for p in tqdm([cifar10_labels[k] for k in cifar10_labels], "eval_images")]
eval_clipscore(images)

eval_images: 100%|██████████████████████████████| 10/10 [00:08<00:00,  1.15it/s]


21.688941955566406

# Train

In [12]:
log_wandb = True
lr = 5e-4
# bs = 64
bs = 896
epochs = 500
# timesteps_training = 10
# timesteps_generate = 10
timesteps_training = 1000
timesteps_generate = 1000
steps_log, steps_eval = 10, 200
# steps_log, steps_eval = 10, 20

splits = list(ds.keys())
data_train, data_val = ds[splits[0]], ds[splits[1]]

steps_epoch = len(data_train) // bs

dataloader = DataLoader(data_train, batch_size=bs, shuffle=True, generator = torch.manual_seed(seed), collate_fn=collate)
optimizer = torch.optim.AdamW(transformer.parameters(), lr=lr)
scheduler.set_timesteps(timesteps_training)

model_size = sum(p.numel() for p in transformer.parameters() if p.requires_grad)
print(f"Model parameters: {model_size / 1e6:.2f}M")
print(f"{len(splits)} splits: {splits}", [len(ds[s]) for s in splits])
assert len(splits)==2 

Model parameters: 156.41M
2 splits: ['train', 'test'] [50000, 10000]


In [13]:
if log_wandb: 
    if wandb.run is not None: wandb.finish()
    wandb.init(project="Hana", name=f"Z-{model_size / 1e6:.2f}M_CIFAR10_LR-{lr}_BS-{bs}_TS-{timesteps_training}_my3090").log_code(".", include_fn=lambda path: path.endswith(".py") or path.endswith(".ipynb") or path.endswith(".json"))

step = 0
last_step_time = time.time()

for _ in range(epochs):
    for labels, latents, prompts_encoded, prompts_atnmask in dataloader:        
        noise = torch.randn_like(latents)
        timesteps = scheduler.timesteps[torch.randint(timesteps_training,(latents.shape[0],))].to(device)
        latents_noisy = scheduler.scale_noise(latents, timesteps, noise)        
        noise_pred = transformer(latents_noisy, prompts_encoded, timesteps, prompts_atnmask).sample
    
        loss = F.mse_loss(noise_pred, noise - latents)
        loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(transformer.parameters(), 1.0)
        optimizer.step()
        optimizer.zero_grad()        
        
        if step > 0 and step % steps_log == 0:
            loss_train = loss.item()
            step_time = (time.time() - last_step_time) / steps_log * 1000
            sample_tp = bs * steps_log / (time.time() - last_step_time)
            print(f"step {step}, epoch: {step / steps_epoch:.4f}, train loss: {loss_train:.4f}, grad_norm: {grad_norm:.2f}, {step_time:.2f}ms/step, {sample_tp:.2f}samples/sec")
            if log_wandb: wandb.log({"loss_train": loss_train, "grad_norm": grad_norm, "step_time": step_time, "step": step, "sample_tp": sample_tp, "sample_count": step * bs, "epoch": step / steps_epoch})
            last_step_time = time.time()
    
        if step % steps_eval == 0:
            transformer.eval()
            loss_eval = eval_loss(data_val)
            scheduler.set_timesteps(timesteps_generate)
            images_eval = [generate(p) for p in tqdm([cifar10_labels[k] for k in cifar10_labels], "eval_images")]
            clipscore = eval_clipscore(images_eval)
            print(f"step {step}, eval loss: {loss_eval:.4f}, clipscore: {clipscore:.2f}")
            if log_wandb: wandb.log({"loss_eval": loss_eval, "clipscore": clipscore, "images_eval": wandb.Image(make_grid(images_eval, 2, 5)), "step": step, "sample_count": step * bs, "epoch": step / steps_epoch})
            transformer.train()
            scheduler.set_timesteps(timesteps_training)
        step += 1

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: g-ronimo to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


eval_images: 100%|██████████████████████████████| 10/10 [01:24<00:00,  8.46s/it]


step 0, eval loss: 11.0687, clipscore: 22.85
step 10, epoch: 0.1818, train loss: 5.2188, grad_norm: 3.45, 10063.24ms/step, 89.04samples/sec
step 20, epoch: 0.3636, train loss: 4.1562, grad_norm: 2.94, 1065.73ms/step, 840.74samples/sec
step 30, epoch: 0.5455, train loss: 3.5469, grad_norm: 2.23, 1087.95ms/step, 823.57samples/sec
step 40, epoch: 0.7273, train loss: 3.3281, grad_norm: 2.41, 1036.00ms/step, 864.87samples/sec
step 50, epoch: 0.9091, train loss: 3.3750, grad_norm: 2.23, 1067.88ms/step, 839.04samples/sec
step 60, epoch: 1.0909, train loss: 3.2188, grad_norm: 1.59, 1017.76ms/step, 880.36samples/sec
step 70, epoch: 1.2727, train loss: 3.1250, grad_norm: 1.51, 1034.63ms/step, 866.01samples/sec
step 80, epoch: 1.4545, train loss: 3.1250, grad_norm: 1.41, 1070.67ms/step, 836.86samples/sec
step 90, epoch: 1.6364, train loss: 3.0781, grad_norm: 1.30, 1029.84ms/step, 870.04samples/sec
step 100, epoch: 1.8182, train loss: 3.1094, grad_norm: 1.26, 1070.81ms/step, 836.75samples/sec
step

eval_images: 100%|██████████████████████████████| 10/10 [01:24<00:00,  8.45s/it]


step 200, eval loss: 3.0406, clipscore: 22.72
step 210, epoch: 3.8182, train loss: 2.8906, grad_norm: 0.99, 9953.49ms/step, 90.02samples/sec
step 220, epoch: 4.0000, train loss: 2.9219, grad_norm: 0.93, 1045.79ms/step, 856.77samples/sec
step 230, epoch: 4.1818, train loss: 2.9844, grad_norm: 0.86, 1052.12ms/step, 851.61samples/sec
step 240, epoch: 4.3636, train loss: 2.9844, grad_norm: 0.86, 1034.02ms/step, 866.52samples/sec
step 250, epoch: 4.5455, train loss: 3.0000, grad_norm: 0.73, 1033.18ms/step, 867.22samples/sec
step 260, epoch: 4.7273, train loss: 2.9844, grad_norm: 0.65, 1065.32ms/step, 841.07samples/sec
step 270, epoch: 4.9091, train loss: 2.9531, grad_norm: 0.89, 1028.46ms/step, 871.21samples/sec
step 280, epoch: 5.0909, train loss: 2.9531, grad_norm: 0.80, 1046.80ms/step, 855.95samples/sec
step 290, epoch: 5.2727, train loss: 2.9062, grad_norm: 0.75, 1065.39ms/step, 841.01samples/sec
step 300, epoch: 5.4545, train loss: 2.9844, grad_norm: 0.77, 1036.11ms/step, 864.77samples

...

step 5000, eval loss: 3.0688, clipscore: 23.24
step 5010, epoch: 91.0909, train loss: 2.2969, grad_norm: 0.63, 9876.78ms/step, 90.72samples/sec
step 5020, epoch: 91.2727, train loss: 2.3438, grad_norm: 0.62, 1023.37ms/step, 875.54samples/sec
step 5030, epoch: 91.4545, train loss: 2.4219, grad_norm: 0.68, 1058.35ms/step, 846.60samples/sec
step 5040, epoch: 91.6364, train loss: 2.3750, grad_norm: 0.66, 1006.60ms/step, 890.13samples/sec
step 5050, epoch: 91.8182, train loss: 2.4062, grad_norm: 0.64, 1062.32ms/step, 843.44samples/sec
step 5060, epoch: 92.0000, train loss: 2.3906, grad_norm: 0.60, 1053.26ms/step, 850.69samples/sec
step 5070, epoch: 92.1818, train loss: 2.3281, grad_norm: 0.61, 1060.74ms/step, 844.69samples/sec
step 5080, epoch: 92.3636, train loss: 2.4062, grad_norm: 0.66, 1062.18ms/step, 843.55samples/sec
step 5090, epoch: 92.5455, train loss: 2.3750, grad_norm: 0.63, 1027.46ms/step, 872.06samples/sec
step 5100, epoch: 92.7273, train loss: 2.3438, grad_norm: 0.60, 1040.39m

eval_images: 100%|██████████████████████████████| 10/10 [01:24<00:00,  8.47s/it]


step 5200, eval loss: 3.2828, clipscore: 23.43
step 5210, epoch: 94.7273, train loss: 2.4062, grad_norm: 0.72, 9968.25ms/step, 89.89samples/sec
step 5220, epoch: 94.9091, train loss: 2.4375, grad_norm: 0.66, 1054.36ms/step, 849.80samples/sec
step 5230, epoch: 95.0909, train loss: 2.3906, grad_norm: 0.64, 1100.11ms/step, 814.46samples/sec
step 5240, epoch: 95.2727, train loss: 2.3906, grad_norm: 0.70, 1054.12ms/step, 850.00samples/sec
step 5250, epoch: 95.4545, train loss: 2.3750, grad_norm: 0.64, 1099.27ms/step, 815.09samples/sec
step 5260, epoch: 95.6364, train loss: 2.3750, grad_norm: 0.63, 1057.50ms/step, 847.28samples/sec
step 5270, epoch: 95.8182, train loss: 2.4531, grad_norm: 0.62, 1038.00ms/step, 863.20samples/sec
step 5280, epoch: 96.0000, train loss: 2.3750, grad_norm: 0.62, 1098.24ms/step, 815.85samples/sec
step 5290, epoch: 96.1818, train loss: 2.3594, grad_norm: 0.62, 1059.06ms/step, 846.03samples/sec
step 5300, epoch: 96.3636, train loss: 2.3438, grad_norm: 0.57, 1100.48m

eval_images:  80%|████████████████████████▊      | 8/10 [01:07<00:16,  8.38s/it]

step 5500, epoch: 100.0000, train loss: 2.3281, grad_norm: 0.60, 1026.71ms/step, 872.69samples/sec
step 5510, epoch: 100.1818, train loss: 2.2344, grad_norm: 0.63, 1063.01ms/step, 842.89samples/sec
step 5520, epoch: 100.3636, train loss: 2.3281, grad_norm: 0.66, 1028.23ms/step, 871.40samples/sec
step 5530, epoch: 100.5455, train loss: 2.3438, grad_norm: 0.66, 1026.66ms/step, 872.73samples/sec
step 5540, epoch: 100.7273, train loss: 2.3594, grad_norm: 0.70, 1060.18ms/step, 845.14samples/sec
step 5550, epoch: 100.9091, train loss: 2.3281, grad_norm: 0.63, 1010.83ms/step, 886.40samples/sec
step 5560, epoch: 101.0909, train loss: 2.3125, grad_norm: 0.64, 1062.99ms/step, 842.91samples/sec
step 5570, epoch: 101.2727, train loss: 2.2500, grad_norm: 0.62, 1028.57ms/step, 871.11samples/sec
step 5580, epoch: 101.4545, train loss: 2.3281, grad_norm: 0.67, 1026.52ms/step, 872.85samples/sec
step 5590, epoch: 101.6364, train loss: 2.3906, grad_norm: 0.65, 1061.20ms/step, 844.33samples/sec


In [14]:
transformer.push_to_hub(f"g-ronimo/hana-alpha11_cifar10_TS-{timesteps_training}_{epochs}e")

diffusion_pytorch_model.safetensors:   0%|          | 0.00/313M [00:00<?, ?B/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [15]:
# !runpodctl remove pod $RUNPOD_POD_ID